## First part

In [8]:
import sys
import seaborn as sns
#!{sys.executable} -m pip install dtw
from dtw import dtw
import librosa, librosa.display, os
import pandas as pd
import numpy as np
from numpy.linalg import norm
from dtw import dtw

path = 'komendy_Kasia/'
signals = list()
dataset_mfcc = list()
labels = open('labels.txt','r').read().split('\n')
set_labels = set(labels)
label_dict = dict()
word_number=dict()
for label in set(labels):
    indices = [i for i, x in enumerate(labels) if x == label]
    label_dict[label] = indices
    word_number[label]=len(indices)
#print(label_dict)
#print(word_number)

In [9]:
feature_dict = dict()
for key in label_dict:
    list_signals = list()
    len_signal = list()
    for el in label_dict[key]:
        filename = "%d.wav"%(el)
        signal, sr = librosa.load(os.path.join(path,filename)) #lista sygnalow
        list_signals.append(signal)
        len_signal.append(len(signal))
    #print(list_signals)
    maks_len = np.max(len_signal)
    mfcc_list = list()
    for signal in list_signals:
        signal = np.pad(signal, (0,maks_len-len(signal)), 'mean')
        mfcc = librosa.feature.mfcc(signal, sr)
        mfcc_list.append(mfcc)
    mean = np.mean( mfcc_list, axis=0 )
    feature_dict[key] = mean
#print(feature_dict)

In [10]:
error = 0

#create result matrix
dataFrame = pd.DataFrame(index = set_labels, columns=set_labels, dtype='float')
dataFrame = dataFrame.fillna(0)

#recognition
for filename in os.listdir('komendy/'):
    y1, sr1 = librosa.load('komendy/'+filename)
    mfcc1 = librosa.feature.mfcc(y1, sr1)
    min_dist = 1000
    for key in feature_dict:
        mfcc2 = feature_dict[key]
        dist, cost, acc_cost, path = dtw(mfcc1.T, mfcc2.T, dist=lambda x, y: norm(x - y, ord=1))
        if dist<min_dist:
            min_dist = dist
            value = key
    index = filename.split('.')[0]
    
    if value == labels[int(index)]:
        print("%s detected correct"%(value))
        dataFrame.loc[labels[int(index)],value]+=1 #count how many times value is recognized as labels[int(index)]
    else:
        print("%s detected incorrect, should be %s"%(value, labels[int(index)]))
        error+=1
        dataFrame.loc[labels[int(index)],value]+=1

print(dataFrame)


RADIO detected incorrect, should be ZAPAL
STOPIEN detected incorrect, should be SWIATLO
WYLACZ detected correct
W detected correct
SYPIALNI detected correct
ZAMKNIJ detected correct
W detected incorrect, should be BRAME
ZWIEKSZ detected correct
LAZIENCE detected incorrect, should be OGRZEWANIE
W detected incorrect, should be O
LAZIENCE detected incorrect, should be JEDEN
STOPIEN detected correct
LAZIENCE detected incorrect, should be ZAKREC
TELEWIZOR detected correct
W detected incorrect, should be WODE
W detected correct
LAZIENCE detected correct
USTAW detected correct
WODE detected incorrect, should be ALARM
PRZYCISZ detected correct
DRZWI detected incorrect, should be RADIO
ZMIEN detected correct
WODE detected incorrect, should be KANAL
WLACZ detected incorrect, should be PODLEJ
TELEWIZOR detected incorrect, should be PODNIES
LAZIENCE detected incorrect, should be KWIATKI
ZAPARZ detected correct
W detected incorrect, should be KAWE
WYLACZ detected correct
GARAZU detected incorrect, 

In [11]:
#macierz procentowa
for i in set_labels:
    for j in set_labels:
        dataFrame.loc[i,j]=dataFrame.loc[i,j]*100/word_number[i]
       
print(dataFrame)
dataFrame.to_excel('results.xls')

            DRZWI  WLACZ  KANAL  ALARM  JEDEN  ZAKREC  WODE  ZAGOTUJ  STOPIEN  \
DRZWI       100.0    0.0    0.0    0.0    0.0     0.0   0.0      0.0      0.0   
WLACZ         0.0   50.0    0.0    0.0    0.0     0.0  25.0      0.0      0.0   
KANAL         0.0    0.0    0.0    0.0    0.0     0.0  75.0      0.0      0.0   
ALARM         0.0   12.5   25.0    0.0    0.0     0.0  25.0      0.0      0.0   
JEDEN        50.0    0.0    0.0    0.0   25.0     0.0   0.0      0.0      0.0   
ZAKREC       25.0    0.0    0.0    0.0    0.0    25.0   0.0      0.0      0.0   
WODE          0.0    0.0    0.0    0.0    0.0     0.0  50.0      0.0      0.0   
ZAGOTUJ       0.0    0.0    0.0    0.0    0.0     0.0   0.0     75.0      0.0   
STOPIEN       0.0    0.0    0.0    0.0    0.0     0.0   0.0      0.0    100.0   
PODNIES      25.0    0.0    0.0    0.0    0.0     0.0   0.0      0.0      0.0   
ZMIEN        25.0    0.0    0.0    0.0    0.0     0.0   0.0      0.0      0.0   
KAWE          0.0    0.0    

In [12]:
cm = sns.light_palette("green", as_cmap=True)

s = dataFrame.style.background_gradient(cmap=cm)
s

,DRZWI,WLACZ,KANAL,ALARM,JEDEN,ZAKREC,WODE,ZAGOTUJ,STOPIEN,PODNIES,ZMIEN,KAWE,WYLACZ,ZAPARZ,LAZIENCE,W,DO,SYPIALNI,ZAMKNIJ,USTAW,ZAPAL,KWIATKI,ZMYWARKE,BRAME,O,PRZYCISZ,KUCHNI,RADIO,GARAZU,ZWIEKSZ,SWIATLO,TELEWIZOR,OTWORZ,OGRZEWANIE,ROLETY,PODLEJ
DRZWI,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
WLACZ,0,50,0,0,0,0,25,0,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
KANAL,0,0,0,0,0,0,75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0
ALARM,0,12.5,25,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,12.5,0,0,0,0,0,0,0
JEDEN,50,0,0,0,25,0,0,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ZAKREC,25,0,0,0,0,25,0,0,0,0,0,0,25,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
WODE,0,0,0,0,0,0,50,0,0,0,0,0,0,0,0,50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ZAGOTUJ,0,0,0,0,0,0,0,75,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
STOPIEN,0,0,0,0,0,0,0,0,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PODNIES,25,0,0,0,0,0,0,0,0,25,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0


In [13]:
success_rate = (172-error)/172*100
print('Success rate: %.2f percent'%(success_rate))

Success rate: 52.91 percent


## Second part: Neural Networks

In [14]:
X = []
y = []
for filename in os.listdir('komendy_Kasia/'):
    y1, sr1 = librosa.load('komendy_Kasia/'+filename)
    mfcc1 = librosa.feature.mfcc(y1, sr1)
    DIST = []
    for key in feature_dict:
        mfcc2 = feature_dict[key]
        dist, cost, acc_cost, path = dtw(mfcc1.T, mfcc2.T, dist=lambda x, y: norm(x - y, ord=1))
        DIST.append(dist)
    X.append(DIST)
    index = filename.split('.')[0]
    val = labels[int(index)]
    y.append(val)


for filename in os.listdir('komendy/'):
    y1, sr1 = librosa.load('komendy/'+filename)
    mfcc1 = librosa.feature.mfcc(y1, sr1)
    DIST = []
    for key in feature_dict:
        mfcc2 = feature_dict[key]
        dist, cost, acc_cost, path = dtw(mfcc1.T, mfcc2.T, dist=lambda x, y: norm(x - y, ord=1))
        DIST.append(dist)
    X.append(DIST)
    index = filename.split('.')[0]
    val = labels[int(index)]
    y.append(val)

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()
clf.fit(X, y)

print(clf.score(X, y))#skuteczność

y_predict = clf.predict(X)
conf = confusion_matrix(y, y_predict)
print(conf)
print(classification_report(y, y_predict))

0.78488372093
[[15  0  0 ...,  0  0  0]
 [ 1  5  0 ...,  0  0  0]
 [ 0  0  6 ...,  0  0  0]
 ..., 
 [ 0  0  0 ...,  8  0  0]
 [ 1  0  0 ...,  0  7  0]
 [ 0  0  0 ...,  0  0  4]]
             precision    recall  f1-score   support

      ALARM       0.62      0.94      0.75        16
      BRAME       0.83      0.62      0.71         8
         DO       0.86      0.75      0.80         8
      DRZWI       0.88      0.88      0.88         8
     GARAZU       0.67      0.88      0.76        16
      JEDEN       0.80      1.00      0.89         8
      KANAL       0.29      0.25      0.27         8
       KAWE       0.60      0.38      0.46         8
     KUCHNI       1.00      0.75      0.86         8
    KWIATKI       0.50      1.00      0.67         8
   LAZIENCE       0.86      0.75      0.80         8
          O       0.80      1.00      0.89         8
 OGRZEWANIE       0.86      0.75      0.80         8
     OTWORZ       0.80      1.00      0.89         8
     PODLEJ       0.78    